In [1]:
#import libraries
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

In [2]:
import os
os.environ["OPENAI_API_KEY"] =""


In [23]:
# print(docs)
model_name = 'gpt-3.5-turbo'
llm = OpenAI(model_name=model_name, temperature=0, max_tokens=2000)

c:\Users\medha\miniconda3\lib\site-packages\langchain\llms\openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\medha\miniconda3\lib\site-packages\langchain\llms\openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [24]:
prompt = '''
generate a csv file that store information about {topic} 
make sure to include the following attriutes {attributes}. use a real word examples.
generate {rows} rows

print the data only without intro or end
'''
prompt_template = PromptTemplate(template=prompt, input_variables=['topic', 'attributes', 'rows'])

res = llm(prompt_template.format(topic="Homes", attributes="Neighborhood, location, bedrooms, bathrooms, house size (sqft), price (k$)", rows="10"))

In [25]:
print(res)

Neighborhood,Location,Bedrooms,Bathrooms,House Size (sqft),Price (k$)
Pacific Heights,San Francisco,4,3,2500,2000
Upper West Side,New York City,3,2,1800,1500
Beverly Hills,Los Angeles,5,4,3500,3000
Back Bay,Boston,2,1,1200,1000
Old Town,Chicago,3,2,2000,1800
Capitol Hill,Seattle,4,3,2200,1900
South Beach,Miami,2,2,1500,1200
Georgetown,Washington D.C.,3,2,1800,1600
River Oaks,Houston,4,4,3000,2500
Cherry Creek,Denver,2,1,1400,1100


In [37]:
with open('home.csv', 'w') as f:
    f.write(res)

In [38]:
import pandas as pd

df = pd.read_csv('home.csv')
df

,Neighborhood,Location,Bedrooms,Bathrooms,House Size (sqft),Price (k$)
0,Pacific Heights,San Francisco,3,2,1800,1500
1,Upper West Side,New York City,2,1,1200,1200
2,Beverly Hills,Los Angeles,4,3,2500,2500
3,Back Bay,Boston,1,1,800,900
4,Old Town,Chicago,2,2,1500,1000
5,Capitol Hill,Seattle,3,2,1800,1300
6,South Beach,Miami,1,1,900,1100
7,Georgetown,Washington D.C.,2,1,1000,950
8,South End,Boston,3,2,1600,1400
9,West Hollywood,Los Angeles,1,1,700,1200


In [3]:
# read and load the csv file that store homes data
loader = CSVLoader(file_path='home.csv')
docs = loader.load()

In [4]:
# load gpt model
model_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(model_name=model_name, temperature=0.3, max_tokens=100)


In [5]:
# create vector store index and query the data
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(documents=split_docs, embedding=embeddings)


In [6]:
# sample questions and answers

personal_questions = [   
    "How big do you want your house to be?" 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",   
]

personal_answers = answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

In [7]:
# create a chat with the customer and summarize it
history = ChatMessageHistory()
history.add_user_message(f"""You are AI sales assisstant that will recommend user a home based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])
    
history.add_ai_message("""Now tell me a summary of a home you're considering in points""")
memory = ConversationSummaryMemory(
    llm=llm,
    chat_memory=history,
    memory_key="chat_history", 
    input_key="question",
    buffer=f"The human answered {len(personal_questions)} personal questions. Use them to extract home attributes like location, price, home area and number of rooms",
    return_messages=True)


In [8]:
# create a prompt
prompt=PromptTemplate(
    template="You are an sales assistant who buy homes. Use the following pieces of retrieved context and customer prefrences to provide the customer with information about available home. Use five sentences maximum and keep the answer attractive. \nContext: {context} \nCustomer's prefernced: {chat_history} \nQuestion: {question}\nAnswer:",
    input_variables=['context', 'chat_history', 'question']
    )
print(prompt)
# create question and answer model to retrieve answers from retrived information
chain_type_kwargs = {'prompt': prompt}

chain = ConversationalRetrievalChain.from_llm(
                                llm=llm,
                                chain_type="stuff",
                                retriever=db.as_retriever(),
                                combine_docs_chain_kwargs=chain_type_kwargs,
                                memory=memory
                            )




input_variables=['context', 'chat_history', 'question'] template="You are an sales assistant who buy homes. Use the following pieces of retrieved context and customer prefrences to provide the customer with information about available home. Use five sentences maximum and keep the answer attractive. \nContext: {context} \nCustomer's prefernced: {chat_history} \nQuestion: {question}\nAnswer:"


In [20]:
# take input from user
query = "as  a sales assisstant, represent the answer in attractive way"

# run the query
result = chain({"question": query})
print(result['answer'])

If you're looking for a charming home in the heart of Georgetown, Washington D.C., we have the perfect option for you! This beautiful house features 2 bedrooms and 1 bathroom, with a spacious house size of 1000 sqft. And the best part? It's priced at just $950k, making it an incredible value for this highly sought-after neighborhood.

If you prefer the bustling city life, our Upper West Side property in New York City might be the one for you.


In [12]:
#%%writefile HomeMatch.py

#import libraries
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory

import os
os.environ["OPENAI_API_KEY"] =""


personal_questions = [   
    "How big do you want your house to be?" 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",   
]


# load gpt model
#model_name = 'gpt-3.5-turbo'
#llm = ChatOpenAI(model_name=model_name, temperature=0.3, max_tokens=100)


# read and load the csv file that store homes data
loader = CSVLoader(file_path='home.csv')
docs = loader.load()


# create vector store index and query the data
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(documents=split_docs, embedding=embeddings)



# create a chat with the customer and summarize it
history = ChatMessageHistory()
history.add_user_message(f"""You are AI sales assisstant that will recommend user a home based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(input('\nanswer: '))
    
history.add_ai_message("""Now tell me a summary of a home you're considering in points""")
memory = ConversationSummaryMemory(
    llm=llm,
    chat_memory=history,
    memory_key="chat_history", 
    input_key="question",
    buffer=f"The human answered {len(personal_questions)} personal questions. Use them to extract home attributes like location, price, home area and number of rooms",
    return_messages=True)

#print(memory.summary_message_cls.content)

# create a prompt
prompt=PromptTemplate(
    template="You are an sales assistant who buy homes. Use the following pieces of retrieved context and customer prefrences to provide the customer with information about available home. Use five sentences maximum and keep the answer attractive. \nContext: {context} \nCustomer's prefernced: {chat_history} \nQuestion: {question}\nAnswer:",
    input_variables=['context', 'chat_history', 'question']
    )

# create question and answer model to retrieve answers from retrived information
chain_type_kwargs = {'prompt': prompt}

chain = ConversationalRetrievalChain.from_llm(
                                llm=llm,
                                chain_type="stuff",
                                retriever=db.as_retriever(),
                                combine_docs_chain_kwargs=chain_type_kwargs,
                                memory=memory
                            )


# take input from user
query = "as  a sales assisstant, represent the answer in attractive way"

# run the query
result = chain({"question": query})
print(result['answer'])


input_variables=['context', 'chat_history', 'question'] template="You are an sales assistant who buy homes. Use the following pieces of retrieved context and customer prefrences to provide the customer with information about available home. Use five sentences maximum and keep the answer attractive. \nContext: {context} \nCustomer's prefernced: {chat_history} \nQuestion: {question}\nAnswer:"
"Imagine living in the prestigious neighborhood of Beverly Hills, right in the heart of Los Angeles. This stunning home boasts 4 spacious bedrooms and 3 luxurious bathrooms, providing ample space for you and your family. With a generous house size of 2500 square feet, you'll have plenty of room to entertain guests and create lifelong memories. And the best part? This dream home is available at an incredible price of just $2500k. Don't miss out on this amazing opportunity to live in


In [11]:
!python HomeMatch.py

^C
